# Final Project SQL

## By Michael Eibner

## Description

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

## Goal

By analyzing this data using SQL queries, we will be able to find out how we can lead more people to reading books, rate them, and writing reviews to suggest it to other people.

# Configuration

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# set the configuration to run SQL queries
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# define a function for execute the queries
def execute(table):
    return pd.io.sql.read_sql(table, con = engine)

In [4]:
# to see the results for the 2nd. task
pd.set_option('display.max_rows', None)

# Tables

In [5]:
# get an overview over the books table (first 5 rows)
books_table = '''
SELECT *
FROM books
LIMIT 5

'''

# get an overview over the authors table (first 5 rows)
authors_table = '''
SELECT *
FROM authors
LIMIT 5

'''

# get an overview over the ratings table (first 5 rows)
ratings_table = '''
SELECT *
FROM ratings
LIMIT 5

'''

# get an overview over the reviews table (first 5 rows)
reviews_table = '''
SELECT *
FROM reviews
LIMIT 5

'''

# get an overview over the publishers table (first 5 rows)
publishers_table = '''
SELECT *
FROM publishers
LIMIT 5

'''

In [6]:
# books table
execute(books_table)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
# authors table
execute(authors_table)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
# ratings table
execute(ratings_table)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
# reviews table
execute(reviews_table)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [10]:
# publishers table
execute(publishers_table)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


# Tasks

## Queries

In [11]:
# task 1
task1 = '''
SELECT COUNT(*) AS number_of_books
FROM books
WHERE publication_date > '2000-01-01'
'''

# task 2
task2 = '''
SELECT b.title,
COUNT(DISTINCT re.review_id) AS number_of_reviews,
AVG(ra.rating) AS average_rating
FROM books AS b
INNER JOIN reviews AS re
USING (book_id)
INNER JOIN ratings AS ra
USING (book_id)
GROUP BY b.title
'''

# tast 3
task3 = '''
SELECT p.publisher, COUNT(b.book_id) AS publications
FROM books AS b
INNER JOIN publishers AS p
USING (publisher_id)
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY publications DESC
LIMIT 1

'''

# task 4
task4 = '''
SELECT a.author, b.title,
AVG(ra.rating) AS average_book_rating
FROM books AS b
INNER JOIN authors AS a
USING (author_id)
INNER JOIN ratings AS ra
USING (book_id)
GROUP BY a.author, b.title
HAVING COUNT(ra.rating_id) > 50
ORDER BY average_book_rating DESC
LIMIT 1
'''

# task 5
task5 = '''
SELECT COUNT(*)/COUNT(DISTINCT username) AS average_number_of_reviews
FROM reviews
WHERE username IN (
SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(*) > 50)

'''

## Task 1

### Find the number of books released after January 1, 2000

The solution contains only the number of all books. So we need only the COUNT function in SELECT, all information from the books table and the dates after 2000-01-01 in the WHERE clause

In [12]:
execute(task1)

,number_of_books
0,819


### Result:

There are 819 books released after 1st. of January 2000

## Task 2

### Find the number of user reviews and the average rating for each book

To see the number of reviews and the average rating of each book, we have to join the reviews, and the ratings tables to the books table. Then a GROUP BY is necessary to group ratings and reviews to the book titles.

In [13]:
execute(task2)

,title,number_of_reviews,average_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
5,1st to Die (Women's Murder Club #1),4,3.500000
6,2nd Chance (Women's Murder Club #2),3,3.000000
7,4th of July (Women's Murder Club #4),3,4.000000
8,A Beautiful Mind,2,4.250000
9,A Bend in the Road,3,3.200000


### Conclusion

By scrolling through the books, it's noticeable that the range of the review count is relatively even. And relatively low. There can be more book reviews to lead more people to read books.

## Task 3

### Identify the publisher that has released the greatest number of books with more than 50 pages

The name of the publisher and the count of their publications is necessary for this task. Additionaly to the books table, we need the publishers table. The task only needs the greatest number, it means, we need only one result with the highest number. We get this by setting a LIMIT and use a ORDER BY with a descending order.

In [14]:
execute(task3)

,publisher,publications
0,Penguin Books,42


## Result

The publisher Penguin Books published the greatest number of books. There were 42 books with more than 50 pages published by Penguin Books

## Task 4

### Identify the author with the highest average book rating

For this task, we need the books and authors tables. But because we will look at books with at least 50 ratings, we need also the ratings table and a HAVING clause. Like in the task before, we only need the hightest average book rating. We get this result with an descending rating order and a LIMIT by 1.

In [15]:
execute(task4)

,author,title,average_book_rating
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634


### Result

J.K. Rowling is the author with the highest average book rating. The highest average book rating is from Harry Potter and the Prisoner of Azkaban with an average rating of 4.41. Mary GrandPré created the cover artwork for the books.

## Task 5

### Find the average number of text reviews among users who rated more than 50 books

At first, we have to find out how many usernames rated more than 50 books and divide the count of all reviews of these users through the count of these users. We will find out the user who rated more than 50 books by using a subquery with a HAVING clause.

In [16]:
execute(task5)

,average_number_of_reviews
0,24


### Result

The average number of reviews of users who rated more than 50 books is 24.

# Conclusion

**The numbers of reviews and ratings are comparatively low, the world needs more of active readers who may lead other people by their opinion about books to reading more books.**